In [3]:
import cv2
import numpy as np
import string
from keras.models import load_model
from collections import defaultdict

# Setup camera
capture = cv2.VideoCapture(0, cv2.CAP_ANY)
capture.set(3, 640)
capture.set(4, 480)

# Define labels
classes = [char for char in string.ascii_uppercase if char != "J" if char != "Z"]

# Load model
model = load_model("ai_model")

while True:
	# Read frame
	ret, frame = capture.read()
	if not ret:
		print("Fail")
		continue
	
	# Process frame
	img = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
	####### Process for threshold (comment out this if you are working with grayscale) ########
	#blur = cv2.GaussianBlur(img,(3,3),2)
	#th = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
	#ret, img = cv2.threshold(th, 0, 255, cv2.THRESH_OTSU)
	###########################################################################################
	out = cv2.resize(img, (128,128), interpolation = cv2.INTER_AREA)
	out = out.reshape(1, 128, 128, 1)

	# Make prediction
	pred = model.predict(out/255)

	# Resize to show bigger display
	img = cv2.resize(img, (1280,960)) # Change to whatever resolution you want (Don't forget to change x and y after)
	
	# Add predictions to a dictionary
	prediction_table = defaultdict()
	for i in range(len(pred[0])):
		pred[0][i] = float("{:.2f}".format(pred[0][i]))
		prediction_table[classes[i]] = int(pred[0][i] * 100)

	# Get predicted label
	pred_label = classes[np.argmax(pred[0])]

	# Put predicted letter and all prediction percentages to display
	x, y = 20, 50
	cv2.putText(img, 'Prediction:{}'.format(pred_label), (x, y), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 3)
	y+=50
	for l in prediction_table:
		cv2.putText(img, '{}:%{}'.format(l, prediction_table[l]), (x, y), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,255,0), 2)
		y+=50
		if y == 700:
			y = 100
			x = 100

	# Show until ESC pressed
	cv2.imshow("Hand Sign Prediction", img)
	k = cv2.waitKey(1)
	if k % 256 == 27:
		break

# Release sources
capture.release()
cv2.destroyAllWindows()